# VGG16 Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
# This will download the model.
# Internet connection is required.
vgg16_model=tf.keras.applications.vgg16.VGG16()

In [3]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Converting VGG16 Model to Sequential Model

In [10]:
model=Sequential()

# We loop through the layers of the VGG16 model and add them to our new model.
# Except the last layer.
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [11]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [12]:
# By Making the trained layers non-trainable, we are freezing the weights of the layers.
# This will prevent the weights from being updated during training.
for layer in model.layers:
    layer.trainable=False

In [13]:
# We are adding our own output layer.
# This is for cats and dogs
model.add(Dense(units=2, activation='softmax'))

# The output layer is the only trainable layer.

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [15]:
model.compile(
    optimizer=Adam(learning_rate=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [16]:
train_path='/Coding/Learning/DL_Learning/Datasets/dogs-vs-cats/train'
valid_path='/Coding/Learning/DL_Learning/Datasets/dogs-vs-cats/valid'
test_path='/Coding/Learning/DL_Learning/Datasets/dogs-vs-cats/test'

In [17]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [18]:
model.fit(
    x=train_batches, 
    validation_data=valid_batches, 
    epochs=5, 
    verbose=2
)

Epoch 1/5
100/100 - 440s - loss: 0.4501 - accuracy: 0.8160 - val_loss: 0.1027 - val_accuracy: 0.9700 - 440s/epoch - 4s/step
Epoch 2/5
100/100 - 529s - loss: 0.1106 - accuracy: 0.9570 - val_loss: 0.0647 - val_accuracy: 0.9700 - 529s/epoch - 5s/step
Epoch 3/5
100/100 - 405s - loss: 0.0678 - accuracy: 0.9770 - val_loss: 0.0478 - val_accuracy: 0.9800 - 405s/epoch - 4s/step
Epoch 4/5
100/100 - 402s - loss: 0.0506 - accuracy: 0.9870 - val_loss: 0.0479 - val_accuracy: 0.9800 - 402s/epoch - 4s/step
Epoch 5/5
100/100 - 573s - loss: 0.0395 - accuracy: 0.9920 - val_loss: 0.0451 - val_accuracy: 0.9700 - 573s/epoch - 6s/step


In [19]:
model.save('VGG16_cats_and_dogs.h5')